In [2]:
"""
YouTube Transcript Retrieval Script
Created by: Hamed Ghane
Date: December 06, 2024

This script retrieves a transcript from a YouTube video using the YouTube Transcript API.
It performs the following steps:
1. Extract the video ID from a given YouTube URL.
2. Use the YouTube Transcript API to request the transcript of that video.
3. Print the transcript to the console if available.

Requirements:
- youtube_transcript_api (Install via: pip install youtube-transcript-api)

Usage:
- Run this script.
- When prompted, enter a valid YouTube URL.
- If a transcript is available, it will be printed to the console.
- If no transcript is available or the URL is invalid, an error message will be displayed.
"""

import re  # Used for regular expressions to extract the video ID from the URL
from youtube_transcript_api import YouTubeTranscriptApi  # Used to fetch YouTube transcripts

def get_video_id(url):
    """
    Extract the YouTube video ID from a given URL using a regular expression.
    Returns the video ID if found, otherwise returns None.
    """
    # Search the provided URL for a pattern that matches a YouTube video ID.
    # The pattern looks for 'v=' or '/' followed by 11 valid characters typical of a YouTube video ID.
    video_id_match = re.search(r"(?:v=|\/)([0-9A-Za-z_-]{11}).*", url)
    
    # If a match is found, return the captured video ID group.
    if video_id_match:
        return video_id_match.group(1)
    
    # If no match is found, return None indicating an invalid or non-matching URL.
    return None

def get_transcript(video_id):
    """
    Retrieve the transcript for the given YouTube video ID using the YouTube Transcript API.
    If successful, returns the transcript as a string. Otherwise, returns None.
    """
    try:
        # Fetch the transcript data from YouTube using the video ID.
        transcript = YouTubeTranscriptApi.get_transcript(video_id)
        
        # The transcript is a list of dictionaries with 'text' keys. 
        # Join all text entries into a single string separated by spaces.
        return " ".join([entry['text'] for entry in transcript])
    
    except Exception as e:
        # If an error occurs (e.g., no transcript found, network issue), print the error and return None.
        print(f"An error occurred: {e}")
        return None

def main():
    """
    Main function of the script:
    1. Prompts the user to input a YouTube URL.
    2. Extracts the video ID from the URL.
    3. Attempts to retrieve the transcript using the obtained video ID.
    4. Prints the transcript if available, or shows an error message otherwise.
    """
    # Prompt the user to enter a YouTube URL.
    url = input("Enter the YouTube URL: ")
    
    # Extract the video ID from the provided URL.
    video_id = get_video_id(url)
    
    # If no valid video ID could be extracted, print an error and stop execution.
    if not video_id:
        print("Invalid YouTube URL")
        return
    
    # Attempt to retrieve the transcript using the extracted video ID.
    transcript = get_transcript(video_id)
    
    # If no transcript could be retrieved, notify the user and stop execution.
    if not transcript:
        print("Failed to retrieve transcript")
        return
    
    # If the transcript retrieval is successful, print it to the console.
    print("\nFull Transcript:")
    print(transcript)

# Run the main function if this file is executed as the main script.
if __name__ == "__main__":
    main()


Enter the YouTube URL:  https://www.youtube.com/shorts/sMtpi3cchNQ



Full Transcript:
here is a hot beverage to comfort you it's in a tog go cup make of that what you will come on it's still early let's do something well I have been toying around with an idea for 4 d chest how about we just talk all right in 4 d chest no come on let's talk about our lives tell me something about you I don't know I own nine pairs of pants okay that that's a good start but I was thinking maybe something a little more personal I see I own nine pairs of Underpants how about I go first but I don't want to know how many Underpants you own although base on the floor of your bedroom I'd say it's a thousand
